In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np

In [2]:
def capture_images():
    cap = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        cv2.imshow('Capture Gesture', (frame))

        key = cv2.waitKey(1)
        if key == ord('c'): 
            image_name = f"{image_prefix}_{count}.jpg"
            image_path = os.path.join(output_dir, image_name)
            cv2.imwrite(image_path, frame)
            print(f"Image {image_name} captured")
            count += 1
        elif key == ord('q') or count >= 100:
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils
def draw_hand_landmarks(frame):
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    return frame

In [4]:
def extract(frame):
    # Load the MediaPipe Hands model
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    lm = np.zeros((2, 21, 3))
    for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
        for j, landmark in enumerate(hand_landmarks.landmark):
            lm[i][j] = [landmark.x, landmark.y, landmark.z]    
    hands.close()

    return lm

In [5]:
def load_dataset(dataset_dir):
    dataset = []
    labels = []

    gesture_names = os.listdir(dataset_dir)
    for gesture_name in gesture_names:
        gesture_dir = os.path.join(dataset_dir, gesture_name)

        if os.path.isdir(gesture_dir):
            for filename in os.listdir(gesture_dir):
                image_path = os.path.join(gesture_dir, filename)
                image = cv2.imread(image_path)

                dataset.append(image)
                labels.append(gesture_name)

    return dataset, labels

In [11]:
dataset_dir = 'dataset'

dataset, labels = load_dataset(dataset_dir)

In [12]:
all_landmarks = []
new_labels = []
deleted = 0
L = 0
for img in dataset:
    try:
        lm = extract(img)
        all_landmarks.append(lm)
        new_labels.append(labels[L])
        L += 1
    except:
        L += 1
        deleted += 1
        continue
    # all_landmarks.append(lm)
landmarks = np.array(all_landmarks)

In [14]:
deleted, landmarks.shape, len(new_labels)

(23, (372, 2, 21, 3), 372)

In [16]:
new_labels = np.array(new_labels)
np.savez('Npz Files/landmarks.npz', arr=landmarks)
np.savez('Npz Files/labels.npz', arr=new_labels)

In [31]:
image_prefix = 'a'
output_dir = 'dataset/three'
capture_images()

Image a_0.jpg captured
Image a_1.jpg captured
Image a_2.jpg captured
Image a_3.jpg captured
Image a_4.jpg captured
Image a_5.jpg captured
Image a_6.jpg captured
Image a_7.jpg captured
Image a_8.jpg captured
Image a_9.jpg captured
Image a_10.jpg captured
Image a_11.jpg captured
Image a_12.jpg captured
Image a_13.jpg captured
Image a_14.jpg captured
Image a_15.jpg captured
Image a_16.jpg captured
Image a_17.jpg captured
Image a_18.jpg captured
Image a_19.jpg captured
Image a_20.jpg captured
Image a_21.jpg captured
Image a_22.jpg captured
Image a_23.jpg captured
Image a_24.jpg captured
Image a_25.jpg captured
Image a_26.jpg captured
Image a_27.jpg captured
Image a_28.jpg captured
Image a_29.jpg captured
Image a_30.jpg captured
Image a_31.jpg captured
Image a_32.jpg captured
Image a_33.jpg captured
Image a_34.jpg captured
Image a_35.jpg captured
Image a_36.jpg captured
Image a_37.jpg captured
Image a_38.jpg captured
Image a_39.jpg captured
Image a_40.jpg captured
Image a_41.jpg captured
Im

In [14]:
# Display the image in a window
cv2.imshow('Image', dataset[51])

# Wait for a key press and close the window when any key is pressed
cv2.waitKey(0)
cv2.destroyAllWindows()